In [12]:
!pip install tensorflow-datasets

In [13]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

In [15]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [16]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=4.0, upper=4.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=4.0, upper = 4.0)
    image = tf.image.random_hue(image, max_delta = 0.8)
    image = tf.image.random_saturation(image, lower=4.0, upper=4.0)
    return image, label

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)        

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [20]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [21]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [22]:
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 11:46:50.233588: {'loss': 1.7680089641094208, 'accuracy': 0.45494375, 'val_loss': 1.0890888560349774, 'val_accuracy': 0.6257}
Epoch 1 ends at 11:51:12.784043: {'loss': 0.8983060778975487, 'accuracy': 0.69785, 'val_loss': 0.9865294041906952, 'val_accuracy': 0.6551}
Epoch 2 ends at 11:55:41.699296: {'loss': 0.5828083561897278, 'accuracy': 0.7997438, 'val_loss': 0.6379766122550722, 'val_accuracy': 0.7816}
Epoch 3 ends at 12:00:11.243132: {'loss': 0.3985716642558575, 'accuracy': 0.8629562, 'val_loss': 0.6952773025081416, 'val_accuracy': 0.7838}
Epoch 4 ends at 12:04:43.756873: {'loss': 0.27522402993142603, 'accuracy': 0.9050375, 'val_loss': 0.6274679491094722, 'val_accuracy': 0.8083}
Epoch 5 ends at 12:09:17.718063: {'loss': 0.20175488320812582, 'accuracy': 0.9304, 'val_loss': 0.6684894033108547, 'val_accuracy': 0.8205}
Epoch 6 ends at 12:13:52.869033: {'loss': 0.15470677706450223, 'accuracy': 0.94685, 'val_loss': 0.7030883520651775, 'val_accuracy': 0.806}
Epoch 7 ends at 1

In [23]:
model.evaluate(test_data)

157/157 [==============================].9878 - accuracy: 0.81 - 0s 63ms/step - loss: 0.9377 - accuracy: 0.82 - 0s 58ms/step - loss: 0.7540 - accuracy: 0.85 - 0s 58ms/step - loss: 0.7451 - accuracy: 0.86 - 0s 57ms/step - loss: 0.7091 - accuracy: 0.87 - 0s 55ms/step - loss: 0.6764 - accuracy: 0.87 - 0s 53ms/step - loss: 0.6699 - accuracy: 0.87 - 0s 51ms/step - loss: 0.8803 - accuracy: 0.86 - 0s 49ms/step - loss: 0.8354 - accuracy: 0.86 - 0s 48ms/step - loss: 0.8536 - accuracy: 0.86 - 1s 48ms/step - loss: 0.8237 - accuracy: 0.86 - 1s 48ms/step - loss: 0.7964 - accuracy: 0.86 - 1s 47ms/step - loss: 0.8030 - accuracy: 0.86 - 1s 48ms/step - loss: 0.7929 - accuracy: 0.86 - 1s 48ms/step - loss: 0.8052 - accuracy: 0.86 - 1s 48ms/step - loss: 0.8462 - accuracy: 0.85 - 1s 47ms/step - loss: 0.8512 - accuracy: 0.85 - 1s 47ms/step - loss: 0.8768 - accuracy: 0.84 - 1s 47ms/step - loss: 0.8580 - accuracy: 0.85 - 1s 46ms/step - loss: 0.8503 - accuracy: 0.85 - 1s 46ms/step - loss: 0.8189 - accuracy: 0.

[0.8697326290569488, 0.8387]

In [24]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))